<a href="https://colab.research.google.com/github/kojiishi/contextual-spacing/blob/master/contextual_spacing_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preparations

In [101]:
!python -V
import numpy as np
import pandas as pd
import re
import unicodedata

!rm -rf contextual-spacing
!git clone https://github.com/kojiishi/contextual-spacing.git

Python 3.6.7
Cloning into 'contextual-spacing'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 3), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (14/14), done.


### Unicode

In [102]:
def u_hex(value):
  hexstr = hex(value)[2:].upper()
  return ('000' + hexstr)[-4:]

def u_name_or_null(c):
  try:
    return unicodedata.name(c)
  except ValueError:
    return None

def u_enc(c, encoding):
  code = 0
  for byte in c.encode(encoding, 'ignore'):
    code = code * 256 + byte
  return u_hex(code) if code else ''
  
def get_unicode_list(range):
  return [[
    u_hex(ord(c)),
    c,
    u_name_or_null(c),
    unicodedata.category(c),
    unicodedata.east_asian_width(c),
    u_enc(c, 'cp932'),
    u_enc(c, 'cp936'),
    u_enc(c, 'cp949'),
    u_enc(c, 'cp950'),
  ] for c in [chr(c) for c in range]]

def get_unicode_df(range):
  df = pd.DataFrame(get_unicode_list(range),
                    index=range,
                    columns=['hex', 'char', 'name', 'gc', 'eaw',
                             'cp932', 'cp936', 'cp949', 'cp950'])
  #df.index.name = 'code'
  return df

r = range(0x20, 0x1FFFF)
u = get_unicode_df(r)
u.head(11)

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950
32,0020,,SPACE,Zs,Na,0020,0020,0020,0020
33,0021,!,EXCLAMATION MARK,Po,Na,0021,0021,0021,0021
34,0022,"""",QUOTATION MARK,Po,Na,0022,0022,0022,0022
35,0023,#,NUMBER SIGN,Po,Na,0023,0023,0023,0023
36,0024,$,DOLLAR SIGN,Sc,Na,0024,0024,0024,0024
37,0025,%,PERCENT SIGN,Po,Na,0025,0025,0025,0025
38,0026,&,AMPERSAND,Po,Na,0026,0026,0026,0026
39,0027,',APOSTROPHE,Po,Na,0027,0027,0027,0027
40,0028,(,LEFT PARENTHESIS,Ps,Na,0028,0028,0028,0028
41,0029,),RIGHT PARENTHESIS,Pe,Na,0029,0029,0029,0029


### Read features.txt from the git repositry, [original](http://blogs.adobe.com/CCJKType/files/2018/04/features.txt) from [Ken's CJK Type blog](https://blogs.adobe.com/CCJKType/2018/04/contextual-spacing.html)

In [103]:
def read_features_txt_as_dict():
  classes = {}
  f = open('contextual-spacing/features.txt')
  for line in f.readlines():
    m = re.search(r'@(\w+) = \[(.+)\];', line)
    if m:
      name = m[1]
      for code_with_lang in m[2].split(' '):
        # `code_with_lang` is like `uniFF1F.cn`.
        # Remove `.cn` suffix and append to `name`.
        code_and_lang = code_with_lang.split('.')
        name_with_lang = name
        if len(code_and_lang) == 2:
          name_with_lang = name_with_lang + '.' + code_and_lang[1]
        # Append the code to the class.
        code = int(code_and_lang[0][3:], 16)
        if name_with_lang in classes:
          classes[name_with_lang].append(code)
        else:
          classes[name_with_lang] = [code]
  f.close()
  return classes

def read_features_txt_as_series():
  classes = read_features_txt_as_dict()
  s = None
  for value, list in classes.items():
    s1 = pd.Series(value, index=list)
    if s is None:
      s = s1
    else:
      s = s.combine(s1, lambda x, y: x if pd.isnull(y) else (y if pd.isnull(x) else str(x)+' '+str(y)))
  return s

u = u.assign(cspc=read_features_txt_as_series())
u.query('cspc == cspc').head(10)

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc
8216,2018,‘,LEFT SINGLE QUOTATION MARK,Pi,A,8165,A1AE,A1AE,A1A5,OpeningBracket OpeningBracketVert.vert
8217,2019,’,RIGHT SINGLE QUOTATION MARK,Pf,A,8166,A1AF,A1AF,A1A6,ClosingBracket ClosingBracketVert.vert
8220,201C,“,LEFT DOUBLE QUOTATION MARK,Pi,A,8167,A1B0,A1B0,A1A7,OpeningBracket OpeningBracketVert.vert
8221,201D,”,RIGHT DOUBLE QUOTATION MARK,Pf,A,8168,A1B1,A1B1,A1A8,ClosingBracket ClosingBracketVert.vert
12289,3001,、,IDEOGRAPHIC COMMA,Po,W,8141,A1A2,A1A2,A142,PeriodComma Centered.tw CenteredVert.tw
12290,3002,。,IDEOGRAPHIC FULL STOP,Po,W,8142,A1A3,A1A3,A143,PeriodComma Centered.tw CenteredVert.tw
12296,3008,〈,LEFT ANGLE BRACKET,Ps,W,8171,A1B4,A1B4,A171,OpeningBracket
12297,3009,〉,RIGHT ANGLE BRACKET,Pe,W,8172,A1B5,A1B5,A172,ClosingBracket
12298,300A,《,LEFT DOUBLE ANGLE BRACKET,Ps,W,8173,A1B6,A1B6,A16D,OpeningBracket
12299,300B,》,RIGHT DOUBLE ANGLE BRACKET,Pe,W,8174,A1B7,A1B7,A16E,ClosingBracket


### [CSS Text 4 Character classes](https://drafts.csswg.org/css-text-4/#text-pacing-classes)

In [104]:
s = pd.Series(dtype=object)
for c in r:
  v = []
  gc = unicodedata.category(chr(c))
  eaw = unicodedata.east_asian_width(chr(c))
  if (gc == 'Ps' and ((c >= 0x3000 and c <=0x303F) or eaw == 'F')) or c in [0x2018, 0x201C]:
    v.append('open')
  if (gc == 'Pe' and ((c >= 0x3000 and c <=0x303F) or eaw == 'F')) or c in [0x2019, 0x201D]:
    v.append('close')
  if c in [0x00B7, 0x2027, 0x30FB]:
    v.append('middle')
  if c in [0xFF1A, 0xFF1B]:
    v.append('colon')
  if c in [0x3001, 0x3002, 0xFF0C, 0xFF0E]:
    v.append('dot')
  if v:
    s.at[c] = ' '.join(v)
u = u.assign(css4=s)
u.query('css4 > ""').head(10)

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc,css4
183,00B7,·,MIDDLE DOT,Po,A,,A1A4,A1A4,A150,NaN,middle
8216,2018,‘,LEFT SINGLE QUOTATION MARK,Pi,A,8165,A1AE,A1AE,A1A5,OpeningBracket OpeningBracketVert.vert,open
8217,2019,’,RIGHT SINGLE QUOTATION MARK,Pf,A,8166,A1AF,A1AF,A1A6,ClosingBracket ClosingBracketVert.vert,close
8220,201C,“,LEFT DOUBLE QUOTATION MARK,Pi,A,8167,A1B0,A1B0,A1A7,OpeningBracket OpeningBracketVert.vert,open
8221,201D,”,RIGHT DOUBLE QUOTATION MARK,Pf,A,8168,A1B1,A1B1,A1A8,ClosingBracket ClosingBracketVert.vert,close
8231,2027,‧,HYPHENATION POINT,Po,A,,,,A145,NaN,middle
12289,3001,、,IDEOGRAPHIC COMMA,Po,W,8141,A1A2,A1A2,A142,PeriodComma Centered.tw CenteredVert.tw,dot
12290,3002,。,IDEOGRAPHIC FULL STOP,Po,W,8142,A1A3,A1A3,A143,PeriodComma Centered.tw CenteredVert.tw,dot
12296,3008,〈,LEFT ANGLE BRACKET,Ps,W,8171,A1B4,A1B4,A171,OpeningBracket,open
12297,3009,〉,RIGHT ANGLE BRACKET,Pe,W,8172,A1B5,A1B5,A172,ClosingBracket,close


### Full list of either CSPC or CSS4 is set

In [105]:
pd.options.display.max_rows = None
u.query('cspc == cspc or css4 == css4')

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc,css4
183,00B7,·,MIDDLE DOT,Po,A,,A1A4,A1A4,A150,NaN,middle
8216,2018,‘,LEFT SINGLE QUOTATION MARK,Pi,A,8165,A1AE,A1AE,A1A5,OpeningBracket OpeningBracketVert.vert,open
8217,2019,’,RIGHT SINGLE QUOTATION MARK,Pf,A,8166,A1AF,A1AF,A1A6,ClosingBracket ClosingBracketVert.vert,close
8220,201C,“,LEFT DOUBLE QUOTATION MARK,Pi,A,8167,A1B0,A1B0,A1A7,OpeningBracket OpeningBracketVert.vert,open
8221,201D,”,RIGHT DOUBLE QUOTATION MARK,Pf,A,8168,A1B1,A1B1,A1A8,ClosingBracket ClosingBracketVert.vert,close
8231,2027,‧,HYPHENATION POINT,Po,A,,,,A145,NaN,middle
12289,3001,、,IDEOGRAPHIC COMMA,Po,W,8141,A1A2,A1A2,A142,PeriodComma Centered.tw CenteredVert.tw,dot
12290,3002,。,IDEOGRAPHIC FULL STOP,Po,W,8142,A1A3,A1A3,A143,PeriodComma Centered.tw CenteredVert.tw,dot
12296,3008,〈,LEFT ANGLE BRACKET,Ps,W,8171,A1B4,A1B4,A171,OpeningBracket,open
12297,3009,〉,RIGHT ANGLE BRACKET,Pe,W,8172,A1B5,A1B5,A172,ClosingBracket,close


### Only in CSS

In [106]:
u.query('cspc != cspc and css4 == css4')

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc,css4
183,00B7,·,MIDDLE DOT,Po,A,,A1A4,A1A4,A150,NaN,middle
8231,2027,‧,HYPHENATION POINT,Po,A,,,,A145,NaN,middle
12318,301E,〞,DOUBLE PRIME QUOTATION MARK,Pe,W,,A895,,A1AA,NaN,close


### Only in features.txt

In [107]:
u.query('cspc == cspc and css4 != css4')

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc,css4
65040,FE10,︐,PRESENTATION FORM FOR VERTICAL COMMA,Po,W,,,,,PeriodCommaVert,NaN
65041,FE11,︑,PRESENTATION FORM FOR VERTICAL IDEOGRAPHIC COMMA,Po,W,,,,,PeriodCommaVert,NaN
65042,FE12,︒,PRESENTATION FORM FOR VERTICAL IDEOGRAPHIC FUL...,Po,W,,,,,PeriodCommaVert,NaN
65043,FE13,︓,PRESENTATION FORM FOR VERTICAL COLON,Po,W,,,,,ColonExclamQuestionVert,NaN
65044,FE14,︔,PRESENTATION FORM FOR VERTICAL SEMICOLON,Po,W,,,,,ColonExclamQuestionVert,NaN
65045,FE15,︕,PRESENTATION FORM FOR VERTICAL EXCLAMATION MARK,Po,W,,,,,ColonExclamQuestionVert,NaN
65046,FE16,︖,PRESENTATION FORM FOR VERTICAL QUESTION MARK,Po,W,,,,,ColonExclamQuestionVert,NaN
65047,FE17,︗,PRESENTATION FORM FOR VERTICAL LEFT WHITE LENT...,Ps,W,,,,,OpeningBracketVert,NaN
65048,FE18,︘,PRESENTATION FORM FOR VERTICAL RIGHT WHITE LEN...,Pe,W,,,,,ClosingBracketVert,NaN
65077,FE35,︵,PRESENTATION FORM FOR VERTICAL LEFT PARENTHESIS,Ps,W,,A6E0,,A15F,OpeningBracketVert,NaN


### Both are set but different values

In [108]:
u.query('cspc == cspc and css4 == css4 and cspc != css4')

,hex,char,name,gc,eaw,cp932,cp936,cp949,cp950,cspc,css4
8216,2018,‘,LEFT SINGLE QUOTATION MARK,Pi,A,8165,A1AE,A1AE,A1A5,OpeningBracket OpeningBracketVert.vert,open
8217,2019,’,RIGHT SINGLE QUOTATION MARK,Pf,A,8166,A1AF,A1AF,A1A6,ClosingBracket ClosingBracketVert.vert,close
8220,201C,“,LEFT DOUBLE QUOTATION MARK,Pi,A,8167,A1B0,A1B0,A1A7,OpeningBracket OpeningBracketVert.vert,open
8221,201D,”,RIGHT DOUBLE QUOTATION MARK,Pf,A,8168,A1B1,A1B1,A1A8,ClosingBracket ClosingBracketVert.vert,close
12289,3001,、,IDEOGRAPHIC COMMA,Po,W,8141,A1A2,A1A2,A142,PeriodComma Centered.tw CenteredVert.tw,dot
12290,3002,。,IDEOGRAPHIC FULL STOP,Po,W,8142,A1A3,A1A3,A143,PeriodComma Centered.tw CenteredVert.tw,dot
12296,3008,〈,LEFT ANGLE BRACKET,Ps,W,8171,A1B4,A1B4,A171,OpeningBracket,open
12297,3009,〉,RIGHT ANGLE BRACKET,Pe,W,8172,A1B5,A1B5,A172,ClosingBracket,close
12298,300A,《,LEFT DOUBLE ANGLE BRACKET,Ps,W,8173,A1B6,A1B6,A16D,OpeningBracket,open
12299,300B,》,RIGHT DOUBLE ANGLE BRACKET,Pe,W,8174,A1B7,A1B7,A16E,ClosingBracket,close
